In [1]:
import json
import csv
import pandas as pd
import sqlite3
import networkx as nx
import matplotlib as plt
import numpy as np

In [2]:
conn = sqlite3.connect('nft_database_simple.db')
c = conn.cursor()

In [3]:
df_all_transactions = pd.read_sql_query("SELECT * from all_transactions", conn)
df_all_transactions.head()

,collection_id,collection_name,token_id,block_timestamp,from_address,to_address
0,6,Cool Cats,0,2021-07-03 17:34:34+00:00,0xfc86a64a8de22cf25410f7601acbd8d6630da93d,0x98bb3a3921200017ebd0af803aeddd464e70e791
1,6,Cool Cats,0,2021-12-31 04:57:23+00:00,0x98bb3a3921200017ebd0af803aeddd464e70e791,0xd9463b55af194d4fa22a532a52f5bdd45b66d290
2,6,Cool Cats,4,2021-07-04 19:37:16+00:00,0x53722a32fddda871f35ef628b252d349744d4b71,0x433c8a73bec1273561e4e2201649de12f20b7d58
3,6,Cool Cats,4,2022-02-14 12:30:16+00:00,0x433c8a73bec1273561e4e2201649de12f20b7d58,0x4d6d847627b96ef00767e0984e10e8e1d884fcf4
4,6,Cool Cats,4,2022-03-07 01:02:23+00:00,0x4d6d847627b96ef00767e0984e10e8e1d884fcf4,0x6d8de2f66efcc3af613e168d98e41769c4357696


In [5]:
df_collections = pd.read_sql_query("SELECT * from collections", conn)
df_collections

,collection_id,collection_hash,collection_name
0,1,0x7bd29408f11d2bfc23c34f18275bbf23bb716bc7,Meebits
1,2,0x4e1f41613c9084fdb9e34e11fae9412427480e56,Terraform
2,3,0xff9c1b15b16263c61d017ee9f65c50e4ae0113d7,Loot
3,4,0xce25e60a89f200b1fa40f6c313047ffe386992c3,dotdotdots
4,5,0x49cf6f5d44e70224e2e23fdcdd2c053f30ada28b,CloneX
5,6,0x1a92f7381b9f03921564a437210bb9396471050c,Cool Cats
6,7,0xed5af388653567af2f388e6224dc7c4b3241c544,Azuki
7,8,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,Cryptopunks
8,9,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,Doodles
9,10,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,Bored Ape Yacht Club


In [10]:
indexes = []

In [12]:
for col in df_collections['collection_name']:
    # get collection
    temp_collection = df_all_transactions[df_all_transactions['collection_name'] == col]
    
    # get total number of transactions
    total_transaction_num = temp_collection.shape[0]
    
    # get total number of distinct wallets
    df_distinct_wallets = temp_collection['from_address'].append(temp_collection['to_address'])
    total_distinct_wallets = df_distinct_wallets.unique().shape[0]
    
    # transaction num by distinct wallet
    df_transaction_num_dist = df_distinct_wallets.value_counts()
    df_transaction_num_dist = df_transaction_num_dist.sort_values(ascending=False)
    df_transaction_num_dist = pd.DataFrame({'wallet_hash':df_transaction_num_dist.index, col:df_transaction_num_dist.values})
    
    # get transaction num distribution (by frequency)
    temp_freq = df_transaction_num_dist
    temp_freq['freq'] = df_transaction_num_dist.value_counts().values
    temp_freq_group = temp_freq.groupby(col).sum('freq')
    temp_freq_group = temp_freq.groupby(col).sum('freq')
    temp_freq_group[col] = temp_freq_group.index
    
    # calculate normalized skewness of transaction distribution
    skewness = 3*(temp_freq_group['freq'].mean()-temp_freq_group['freq'].mode()[0])/temp_freq_group[col].std()
    
    # calculate abnormal index
    index = total_transaction_num/(total_distinct_wallets*skewness)
    
    # append index
    indexes.append(index)

indexes

[1.820205177862153,
 12.941687839311061,
 5.462384895387752,
 0.5713039710297361,
 0.41540145726470473,
 1.2569444478951848,
 0.2565670284119814,
 16.269809768716737,
 3.6822703999294553,
 1.939158963311821,
 0.47536107587261506,
 0.4248281183907641,
 1.2537585950703072,
 0.6612967746465936]

In [15]:
df_result = pd.DataFrame({'collection_id': df_collections['collection_id'],
                          'collection_hash':df_collections['collection_hash'],
                          'collection_name':df_collections['collection_name'],
                          'abnormal_index':indexes})
df_result

,collection_id,collection_hash,collection_name,abnormal_index
0,1,0x7bd29408f11d2bfc23c34f18275bbf23bb716bc7,Meebits,1.820205
1,2,0x4e1f41613c9084fdb9e34e11fae9412427480e56,Terraform,12.941688
2,3,0xff9c1b15b16263c61d017ee9f65c50e4ae0113d7,Loot,5.462385
3,4,0xce25e60a89f200b1fa40f6c313047ffe386992c3,dotdotdots,0.571304
4,5,0x49cf6f5d44e70224e2e23fdcdd2c053f30ada28b,CloneX,0.415401
5,6,0x1a92f7381b9f03921564a437210bb9396471050c,Cool Cats,1.256944
6,7,0xed5af388653567af2f388e6224dc7c4b3241c544,Azuki,0.256567
7,8,0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb,Cryptopunks,16.269810
8,9,0x8a90cab2b38dba80c64b7734e58ee1db38b8992e,Doodles,3.682270
9,10,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,Bored Ape Yacht Club,1.939159


In [17]:
df_result.to_sql("connections", conn, if_exists='replace', index=False)